### Imports

In [ ]:
import os
os.chdir(os.getcwd() + '/..')
print(os.getcwd())

import data.data as data
import inout.importexport as imp
import numpy as np
import pandas as pd
import xgboost as xgb
import random
import math


### Create the complete dataframe

#### Read the recommendations from csv

In [ ]:
raw_recs = imp.importcsv('submission/13-12-2018/gxboost25recommendations_12-17-23.csv', check_len=-1)

print(raw_recs[0:2])

#### Explode each row into multiple rows (one per interaction)

In [ ]:
recs_tracks = []
for rec in raw_recs:
    playlist_id = rec[0]
    for t in rec[1:]:
        recs_tracks.append([playlist_id, t])
recs_df = pd.DataFrame(recs_tracks, columns=['playlist_id','track_id'])

print(recs_df)

#### Append the 'profile_length' column to the recommendation dataframe

In [ ]:
target_ids = data.get_target_playlists()
targetURM = data.get_urm_train_1()[target_ids]
user_profile_lengths = np.array(targetURM.sum(axis=1)).flatten()
profile_lengths_df = pd.DataFrame({'playlist_id': target_ids, 'profile_length': user_profile_lengths})

print(profile_lengths_df.head(10))

In [ ]:
rec_lengths_df = recs_df.merge(profile_lengths_df, on='playlist_id')
print(rec_lengths_df)

#### Popularity feature

In [ ]:
df = data.get_playlists_df()
popularity = df.groupby(['track_id']).size().reset_index(name='popularity')
print(popularity)

In [ ]:
rec_pop_df = rec_lengths_df.join(popularity.set_index('track_id'), on='track_id')
print(rec_pop_df)

#### Append the 'label' column 

In [ ]:
urm_test = data.get_urm_test_1()
test_labels = []

last_playlist_id = -1
for idx,row in recs_df.iterrows():
    current_playlist_id = row['playlist_id']
    track_id = row['track_id']
    # cache the row of the urm test if same playlist of the previous iteration
    if not current_playlist_id == last_playlist_id:
        # tracks ids in the t row of urm test
        tracks_ids = urm_test.getrow(current_playlist_id).nonzero()[1]
        last_playlist_id = current_playlist_id
    
    test_labels.append(1 if track_id in tracks_ids else 0)

test_labels_df = pd.DataFrame({'label': test_labels})

In [ ]:
rec_label_df = pd.concat([rec_pop_df, test_labels_df], axis=1)
print(rec_label_df)

### I'm happy with the features gathered

In [ ]:
feature_df = rec_label_df

### Split into train and test dataframes

In [ ]:
def func(x):
    n = x['label'].sum()
    ones = x.loc[x['label'] == 1]
    zeros = x.loc[x['label'] == 0].sample(n)
    return pd.concat([ones,zeros])

In [ ]:
full = feature_df.groupby(['playlist_id'], as_index=False).apply(func)

In [ ]:
full = full.reset_index().drop(['level_0', 'level_1'], axis=1)

In [ ]:
import data.data as d

tgt = data.get_target_playlists()
train_tgt = random.sample(tgt, math.floor(len(tgt)*0.8))
test_tgt = list(set(tgt) - set(train_tgt))
train = full.loc[full['playlist_id'].isin(train_tgt)]
test = full.loc[full['playlist_id'].isin(test_tgt)]

In [ ]:
print(train)

In [ ]:
print(test)

#### One hot encodings

In [ ]:
to_concat_train = []
to_concat_test = []

In [ ]:
oh_playlist_train = pd.get_dummies(train['playlist_id'], prefix='p').to_sparse(fill_value=0)
oh_playlist_test = pd.get_dummies(test['playlist_id'], prefix='p').to_sparse(fill_value=0)
to_concat_train.append(oh_playlist_train)
to_concat_test.append(oh_playlist_test)
train = train.drop(['playlist_id'], axis=1)
test = test.drop(['playlist_id'], axis=1)

In [ ]:
oh_track_train = pd.get_dummies(train['track_id'], prefix='t').to_sparse(fill_value=0)
oh_track_test = pd.get_dummies(test['track_id'], prefix='t').to_sparse(fill_value=0)
to_concat_train.append(oh_track_train)
to_concat_test.append(oh_track_test)
train = train.drop(['track_id'], axis=1)
test = test.drop(['track_id'], axis=1)

In [ ]:
to_concat_train.insert(0, train)
to_concat_test.insert(0, test)

In [ ]:
train = pd.concat(to_concat_train, axis=1)
test = pd.concat(to_concat_test, axis=1)

In [ ]:
print(train)

In [ ]:
print(test)

### Train with XGBoost

In [ ]:
dtrain = xgb.DMatrix(oh1, [])